In [2]:
import pandas as pd
import folium as fl

In [3]:
bps = pd.read_csv(
    'assets/lido_boundary_points.csv'
).dropna(how='all')

bs = pd.read_csv(
    'assets/lido_boundaries.csv'
)

ta = pd.read_csv(
    'assets/lido_trade_areas.csv',
)

bs = (bs
 .merge(ta, how='inner', left_on='TradeAreaId', right_on='Id')
 .drop(['Id_y', 'Rank', 'BoundaryType'], axis='columns')
 .rename(columns={'Id_x': 'BoundaryId'}))

bps = (bps
 .merge(bs, how='inner', left_on='BoundaryId', right_on='BoundaryId')
 .drop(['TradeAreaId'], axis=1))

del ta

In [4]:
grp_stores = bps.sort_values(['BoundaryId', 'Order']).groupby('StoreName')

# Plot a single store

In [8]:
target_store = grp_stores.get_group('Banawe')
store_boundaries = target_store.groupby('BoundaryId')
m = fl.Map(location=target_store.iloc[0][['Latitude', 'Longitude']].tolist(), zoom_start=13)

for _, g in store_boundaries:
    coords = g.sort_values('Order')[['Latitude', 'Longitude']].values.tolist()
    fl.PolyLine(coords, color='blue', weight=2.5, opacity=1).add_to(m)

m

# Plot All Stores

In [6]:
center = grp_stores.get_group('Pasig').iloc[0][['Latitude', 'Longitude']].tolist()
m = fl.Map(location=center, zoom_start=11)

for key, g in grp_stores:
    target_store = grp_stores.get_group(key)
    store_boundaries = target_store.groupby('BoundaryId')
    for _, g in store_boundaries:
        coords = g.sort_values('Order')[['Latitude', 'Longitude']].values.tolist()
        fl.PolyLine(coords, color='blue', weight=2.5, opacity=1).add_to(m)

m